In [55]:
import pandas as pd
import numpy as np
import random
import csv
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix  
import matplotlib.pyplot as plt

In [57]:
from __future__ import division
import json
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression

In [63]:
from sklearn import datasets, metrics, model_selection

X, Y = datasets.make_classification(n_samples=80, n_features=10)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.2)

In [68]:
#Y
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9
0,-2.440440,1.228015,0.839140,0.747243,-2.332011,-1.025721,-0.309797,0.411323,-0.049902,-0.962579
1,-0.739745,0.333685,-0.197170,-0.295583,-0.610100,0.524435,-1.410988,-0.075167,-0.031441,-0.492415
2,-2.581916,-0.010949,1.019422,0.486643,-2.495416,-1.328715,1.686936,-0.121203,0.291490,-0.417412
3,-1.535876,2.356874,0.963426,1.560413,-1.560941,-1.450889,1.774207,-1.786244,0.493295,-0.008363
4,1.025944,-0.281376,0.228430,0.874686,0.855791,-0.644039,0.680662,-1.684622,-0.197545,-2.050777
...,...,...,...,...,...,...,...,...,...,...
75,-1.400792,1.091927,0.660738,-0.070337,-1.376937,-0.920058,1.396267,-0.694477,-0.661144,0.170899
76,0.639514,1.293128,-0.301240,0.351279,0.628536,0.419279,0.726491,0.062272,-0.532178,-0.386927
77,1.057271,0.962890,-1.243306,-0.234774,1.198860,2.071980,0.355079,0.005200,-1.620087,-0.403837
78,1.275419,0.281543,-0.290058,-1.173632,1.186925,0.261343,0.308281,-0.883204,-0.176527,0.480768


In [69]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [70]:
# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'num_leaves': 63,
	'num_trees': 100,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [71]:
# number of leaves,will be used in feature transformation
num_leaf = 63


print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=lgb_train)

print('Save model...')
# save model to file
gbm.save_model('model.txt')

print('Start predicting...')
# predict and get data on leaves, training data
y_pred = gbm.predict(X_train,pred_leaf=True)

Start training...
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000312 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	training's binary_logloss: 0.684793
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	training's binary_logloss: 0.676263
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	training's binary_logloss: 0.668238
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	training's binary_logloss: 0.660034
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	training's binary_logloss: 0.652319
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[6]	training's binary_logloss: 0.644965
[LightGBM] [Warning] No further splits with positive gain, best

/Users/yangweiliang/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [72]:
y_pred.shape

(64, 100)

In [75]:
y_pred

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32)

In [86]:
# feature transformation and write result
print('Writing transformed training data')
transformed_training_matrix = np.zeros([len(y_pred),len(y_pred[0]) * num_leaf],dtype=np.int64)
transformed_training_matrix.shape

Writing transformed training data


(64, 6300)

In [87]:
np.arange(len(y_pred[0])) * num_leaf - 1 + np.array(y_pred[0])

array([   0,   63,  126,  189,  252,  315,  378,  441,  504,  567,  630,
        693,  756,  819,  882,  945, 1008, 1071, 1134, 1197, 1260, 1323,
       1386, 1449, 1512, 1575, 1638, 1701, 1764, 1827, 1890, 1953, 2016,
       2079, 2142, 2205, 2268, 2331, 2394, 2457, 2520, 2583, 2646, 2709,
       2772, 2835, 2898, 2961, 3024, 3087, 3150, 3213, 3276, 3339, 3402,
       3465, 3528, 3591, 3654, 3717, 3780, 3843, 3906, 3969, 4032, 4095,
       4158, 4221, 4284, 4347, 4410, 4473, 4536, 4599, 4662, 4725, 4788,
       4851, 4914, 4977, 5040, 5103, 5166, 5229, 5292, 5355, 5418, 5481,
       5544, 5607, 5670, 5733, 5796, 5859, 5922, 5985, 6048, 6111, 6174,
       6237])

In [88]:
np.array(y_pred[0])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [89]:
for i in range(0,len(y_pred)):
	temp = np.arange(len(y_pred[0])) * num_leaf - 1 + np.array(y_pred[i])
	transformed_training_matrix[i][temp] += 1

#for i in range(0,len(y_pred)):
#	for j in range(0,len(y_pred[i])):
#		transformed_training_matrix[i][j * num_leaf + y_pred[i][j]-1] = 1
transformed_training_matrix

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [90]:
print('Calculate feature importances...')
# feature importances
print('Feature importances:', list(gbm.feature_importance()))
print('Feature importances:', list(gbm.feature_importance("gain")))


# Logestic Regression Start
print("Logestic Regression Start")

# load or create your dataset
print('Load data...')

Calculate feature importances...
Feature importances: [85, 0, 0, 0, 15, 0, 0, 0, 0, 0]
Feature importances: [1882.1921005249023, 0.0, 0.0, 0.0, 251.7265911102295, 0.0, 0.0, 0.0, 0.0, 0.0]
Logestic Regression Start
Load data...


In [94]:
c = np.array([1,0.5,0.1,0.05,0.01,0.005,0.001])
for t in range(0,len(c)):
	lm = LogisticRegression(penalty='l2',C=c[t]) # logestic model construction
	lm.fit(transformed_training_matrix,y_train)  # fitting the data

	#y_pred_label = lm.predict(transformed_training_matrix )  # For training data
	#y_pred_label = lm.predict(transformed_testing_matrix)    # For testing data
	#y_pred_est = lm.predict_proba(transformed_training_matrix)   # Give the probabilty on each label
	y_pred_est = lm.predict_proba(transformed_training_matrix)   # Give the probabilty on each label